In [10]:
# 必要なPythonライブラリ
import sortedcontainers
from sortedcontainers import SortedDict # ソートされた辞書型データ構造を提供するライブラリ
import random
from sage.misc.sage_timeit import sage_timeit
import time

In [11]:
# 整数をnnビットのバイナリ文字列に変換する
int_to_bin = lambda x, nn: format(x, 'b').zfill(nn)

# バイナリ文字列を再度整数に変換する
def bin_to_int(b):
    out = 0
    bp = b[::-1]
    for i in range(len(bp)):
        # ビット列を逆順にしてから計算
        out += ZZ(bp[i])*2**i
    return out

# nnビットのビットリバーサル置換を行う
def bit_reverse(k, nn):
    bp = int_to_bin(k, nn)[::-1]
    return bin_to_int(bp)

# ベクトルの最初のエントリを範囲1から(q-1)/2に収める
def get_sign(vec):
    i = 0
    while vec[i] == 0 and i < len(vec) - 1:
        i += 1
    if ZZ(vec[i]) > (q - 1) / 2:
        return -1
    else:
        return 1

def sign_assign(vec):
    if get_sign(vec) == -1:
        return [-vec[_] for _ in range(len(vec))]
    else:
        return vec

def is_zero(vec):
    i = 0
    while vec[i] == 0 and i < len(vec) - 1:
        i += 1
    if vec[i] == Mod(0, q):
        return True
    else:
        return False

# ビットリバーサル置換とゼータ操作を行うクラスを作成
class bitZeta():
    def __init__(self, N, B, q):
        self.N = N
        self.nn = 2**N
        # self.kB = kB
        self.B = B
        self.tnn = self.nn * 2
        # ビットリバーサルの結果を格納
        self.bit_reversal_lookup = [bit_reverse(ZZ(Mod(i, self.nn)), self.N) for i in range(self.tnn)]
        # 符号変換テーブルの作成
        self.sign_lookup = [[Mod((-1) ** ((ZZ(Mod(i - h, self.nn)) - (i - h)) / self.nn), q) for i in range(self.nn)] for h in range(self.tnn)]

    # ゼータ操作をプライオリティベースに適用するが、最初のブロックのみに適用する関数
    def bzeta(self, vec, h, ii):
        return [vec[self.bit_reversal_lookup[self.bit_reversal_lookup[i] - h]] * self.sign_lookup[h][self.bit_reversal_lookup[i]] for i in range(self.B * (ii - 1), self.B * ii)]

    ## ビット反転によって位数が高い順にベクトルを並び替えた後、回転を行う
    def zeta(self, vec, h):
        return [vec[self.bit_reversal_lookup[self.bit_reversal_lookup[i] - h]] * self.sign_lookup[h][self.bit_reversal_lookup[i]] for i in range(self.nn)]

In [12]:
# BKW Reductionアルゴリズムの基底クラス
class BKW:

    def __init__(self, q, kn, B, sample_input_list, num_samps, alldiffs=False):
        self.q = q
        self.kn = kn # 次元
        self.B = B # ブロックサイズ
        self.n = 2**self.kn
        self.nB = len(self.B)
        self.sample_input_list = sample_input_list
        self.num_samps = num_samps
        self.tables = [SortedDict([]) for _ in range(self.nB + 1)] # BKWテーブルの数を定義
        self.passcount = 0 # サンプルの差分が次のテーブルに渡された回数をカウント
        ## ここコード分けて変更
        self.kB = log(self.B[self.nB-1], 2)
        self.bit = bitZeta(self.kn, self.kB, self.q) # ビットリバーサルおよびゼータ操作のセットアップ
        if alldiffs:
            self.table_insert_blind = self.table_insert_blind_alldiffs
        else:
            self.table_insert_blind = self.table_insert_blind_onediff
        return
    '''
    def reduce_final_table(self):
        # 最終テーブルから重複を取り除くアルゴリズム
        finaltable = SortedDict([])
        for samp in self.tables[int(self.nB)]:
            finaltable[samp] = self.tables[int(self.nB)][samp]
        for key in finaltable: # ループを実行
            samp = self.tables[int(self.nB)][key][0]
            start = self.n - self.B[self.nB-1] #ここなんで-1してたんだっけ？→配列のインデックスやから。
            end = self.n
            # 回転のリストを収集
            rotations = SortedDict([])
            for j in range(self.B[self.nB - 1]):
            ##############   ここおかしいかも
                samp1 = self.bit.zeta(samp, j * int(self.nB-1)) # ここなんで-1するとうまく行くのかわからない
            ##############  ここおかしいかも->うまくいってそうだけど、ブロック数が同じ時とちょっと整合性が取れていないので確認を取りたい
                samp1abs = sign_assign(samp1)
                rotations[repr(samp1abs[start:end])] = samp1abs
            # peekitemはその行列の最後尾を取り出す
            mysamp = rotations.peekitem(0)[1]
            myrep = rotations.peekitem(0)[0]
            key = repr(samp[start:end])
            if key in self.tables[self.nB]:
              self.tables[int(self.nB)].pop(key)
            if myrep not in self.tables[int(self.nB)]:
                self.tables[int(self.nB)][myrep] = [mysamp]
        return
        '''
    def reduce_final_table(self):
        """
        最終テーブルの重複を取り除くアルゴリズム（ブロック数がステップごとに異なる場合対応）
        chatGPT参照->動いたけど参照したい
        """
        # 最終テーブルを複製
        finaltable = SortedDict([])
        for samp in self.tables[int(self.nB)]:
            finaltable[samp] = self.tables[int(self.nB)][samp]  # finaltableに複製

        # 各サンプルについて処理
        for key in finaltable:
            samp = self.tables[int(self.nB)][key][0]

            # 動的に開始インデックスと終了インデックスを計算
            cumulative_block_sizes = [sum(self.B[:i]) for i in range(1, len(self.B) + 1)]
            start = cumulative_block_sizes[-2]  # 最後から2つ目のブロックの開始位置
            end = cumulative_block_sizes[-1]   # 最終ブロックの終了位置

            # 回転のリストを収集
            rotations = SortedDict([])
            block_size = self.B[self.nB - 1]  # 現在のステップのブロックサイズ

            for j in range(block_size):
                # 動的に回転量を計算
                rotation_amount = j * block_size
                samp1 = self.bit.zeta(samp, rotation_amount)  # 回転
                samp1abs = sign_assign(samp1)  # 符号を正規化
                rotations[repr(samp1abs[start:end])] = samp1abs

            # 最小の回転結果を選択
            mysamp = rotations.peekitem(0)[1]
            myrep = rotations.peekitem(0)[0]

            # テーブルを更新
            key = repr(samp[start:end])
            if key in self.tables[self.nB]:
                self.tables[int(self.nB)].pop(key)  # 重複を削除
            if myrep not in self.tables[int(self.nB)]:
                self.tables[int(self.nB)][myrep] = [mysamp]  # 最小の正規形を保存

        return


    def report(self):
        # サンプル削減後の基本情報を報告
        self.reduce_final_table()

        # テーブル間のパス数を報告
        print("Number of times a sample was passed to another table:", self.passcount)

        # テーブルサイズを報告
        totalsizes = 0
        for i in range(1, len(self.tables)):
            print("Table", i, "has", len(self.tables[i]), "entries.")
            if i < len(self.tables) - 1:
                totalsizes += len(self.tables[i])
        print("Total stored table rows (not counting final table):", totalsizes)

        return

    def show_final(self):
        # 最終テーブルを表示
        for i in range(len(self.tables[int(self.nB)])):
            samplist = self.tables[int(self.nB)].peekitem(i)[1]
            for samp in samplist:
                print("           "+str(samp))

    # 1つの差分のみを次に渡すTraditional BKWのテーブル挿入
    def table_insert_blind_onediff(self, samp, i):
        table = self.tables[i]
        samp1 = sign_assign(samp) # 必要に応じて符号を変更
        # 衝突があり、かつ最後のテーブルでない場合は差分を渡す
        start = 0
        for t in range(i-1):
            start += self.B[t]
        rep1 = repr(samp1[start:start + self.B[i-1]])
        if i < self.nB and rep1 in table:
            tabvec = table[rep1][0] # サンプルリストから1つを取り出す
            diff = list(vector(samp1) - vector(tabvec))
            if not is_zero(diff): # 非ゼロの場合、次のテーブルに送る
                self.passcount += 1
                self.table_insert_blind(diff, i + 1)
        else: # 衝突がなく、または最後のテーブルの場合、そのまま格納
            table[rep1] = [samp1] # サンプルをリストとして格納
        return

    # すべての差分を渡すTraditional BKWのテーブル挿入
    def table_insert_blind_alldiffs(self, samp, i):
        table = self.tables[i]
        samp1 = sign_assign(samp) # 必要に応じて符号を変更
        # 衝突があり、かつ最後のテーブルでない場合は差分を渡す
        rep1 = repr(samp1[self.B[i-1] * (i - 1):self.B[i-1] * i])
        if i < self.nB+1 and rep1 in table:
            tabvecs = table[rep1] # サンプルリストを取得
            for tabvec in tabvecs: # 各差分を次に渡す
                diff = list(vector(samp1) - vector(tabvec))
                if not is_zero(diff): # 非ゼロの場合、次のテーブルに送る
                    self.passcount += 1
                    self.table_insert_blind(diff, i + 1)
            table[rep1].append(samp1) # 新しいサンプルも格納
        else: # 衝突がなく、または最後のテーブルの場合、そのまま格納
            table[rep1] = [samp1] # サンプルをリストとして格納
        return

In [13]:
# Traditional BKWを完全にリングブラインドで実行（回転を使用しない）
class blind_BKW(BKW):
    def __init__(self, q, kn, B, sample_input_list, num_samps, alldiffs=False):
        super().__init__(q, kn, B, sample_input_list, num_samps, alldiffs)

    def run(self):
        for s in range(self.num_samps): # 各サンプルを最初のテーブルに渡す
            samp = self.sample_input_list[s]
            self.table_insert_blind(samp, 1)

In [14]:
# サンプルとその回転を使用するTraditional BKW
class trad_BKW(BKW):
    def __init__(self, q, kn, B, sample_input_list, num_samps, alldiffs=False):
        super().__init__(q, kn, B, sample_input_list, num_samps, alldiffs)

    def run(self):
        for s in range(self.num_samps):
            samp = self.sample_input_list[s]
            self.table_insert_blind(samp, 1)
            for j in range(1, self.n): # サンプルの全回転を最初のテーブルに入力
                samp1 = self.bit.zeta(samp, j)
                self.table_insert_blind(samp1, 1)

In [15]:
# Advanced Keying BKW
class adv_BKW(BKW):
    def __init__(self, q, kn, B, sample_input_list, num_samps, alldiffs=False):
        super().__init__(q, kn, B, sample_input_list, num_samps, alldiffs)
        if alldiffs:
            self.table_insert_adv = self.table_insert_adv_alldiffs
        else:
            self.table_insert_adv = self.table_insert_adv_onediff
        return

    def get_rotations(self, samp, i):
        start = self.B[i-1] * (i - 1)
        end = self.B[i-1] * i
        # 回転リストを収集
        rotations = SortedDict([])
        samp1abs = sign_assign(samp)
        repr1abs = repr(samp1abs[start:end])
        if repr1abs not in rotations:
            rotations[repr1abs] = [[samp1abs[start:end], 0]] # どの回転かを格納
        else:
            rotations[repr1abs].append([samp1abs[start:end], 0])
        for j in range(1, self.B):
            samp1 = self.bit.bzeta(samp, j * int(self.nB), i) # ブロック内でのみ回転を計算
            samp1abs = sign_assign(samp1)
            repr1abs = repr(samp1abs)
            if repr1abs not in rotations:
                rotations[repr1abs] = [[samp1abs, j]]
            else:
                rotations[repr1abs].append([samp1abs, j])
        # キャノニカルエントリを探す
        myrep = rotations.peekitem(0)[0] # キャノニカルな代表
        mysamps = rotations.peekitem(0)[1] # すべてのキャノニカルサンプル
        # キャノニカルなサンプルに対してのみフル回転を実行
        for sampy in mysamps:
            samp2 = sign_assign(self.bit.zeta(samp, sampy[1] * self.nB))
            sampy.append(samp2) # フル回転を格納
        return myrep, mysamps

    def table_insert_adv_onediff(self, samp, i):
        table = self.tables[i]
        start = self.B[i-1] * (i - 1)
        end = self.B[i-1] * i
        # 最後のテーブルでない場合
        if i < self.nB:
            myrep, mysamps = self.get_rotations(samp, i)
            # 回転を順次処理
            if myrep in table: # 衝突がある場合
                tabvecs = table[myrep] # その行にあるすべてのベクトル
                mysamp = mysamps[0][2] # フル回転
                for tabvec in tabvecs: # 各古いサンプルに対して
                    diff = list(vector(mysamp) - vector(tabvec)) # 差分を計算
                    if not is_zero(diff): # 非ゼロの場合、次のテーブルに送る
                        self.passcount += 1
                        self.table_insert_adv(diff, i + 1)
            else: # テーブルにまだ存在しない場合
                table[myrep] = [mysamps[0][2]] # 最初のキャノニカルサンプルをその行に格納
        else: # ただの最終テーブルなら、そのまま格納
            samp1abs = sign_assign(samp)
            table[repr(samp1abs[start:end])] = [samp1abs]
        return

    def table_insert_adv_alldiffs(self, samp, i):
        table = self.tables[i]
        start = self.B[i-1] * (i - 1)
        end = self.B[i-1] * i
        # 最後のテーブルでない場合
        if i < self.nB:
            myrep, mysamps = self.get_rotations(samp, i)
            # キャノニカルサンプルを比較して渡す
            if myrep not in table: # 新しい代表の場合、格納
                table[myrep] = [mysamps[0][2]] # 最初の1つを格納
                mysamps.pop(0) # 最初のエントリを削除
            # この時点でテーブルに何かが存在する
            for mysamp in mysamps: # 各新しいサンプルに対して
                for tabvec in table[myrep]: # 各古いサンプルに対して
                    diff = list(vector(mysamp[2]) - vector(tabvec)) # 差分を計算
                    if not is_zero(diff): # 非ゼロの場合、次のテーブルに送る
                        self.passcount += 1
                        self.table_insert_adv(diff, i + 1)
                table[myrep].append(mysamp[2]) # 新しいサンプルをその行に格納
        else: # ただの最終テーブルなら、そのまま格納
            samp1abs = sign_assign(samp)
            table[repr(samp1abs[start:end])] = [samp1abs]
        return

    def run(self):
        for s in range(self.num_samps):
            samp = self.sample_input_list[s]
            self.table_insert_adv(samp, 1)
            for j in range(1, int(self.nB)): # 各サンプルを0,1,...,n/B-1で回転させて最初のテーブルに渡す
                samp1 = self.bit.zeta(samp, j)
                self.table_insert_adv(samp1, 1)
        return

In [16]:
blindfalse = None
blindtrue = None
tradfalse = None
tradtrue = None
advfalse = None
advtrue = None

def run_experiment(q, kn, B, numsamps, alldiffs=False, seed=None, show_final=False):
    global blindfalse
    global blindtrue
    global tradfalse
    global tradtrue
    global advfalse
    global advtrue

    # パラメータの設定

    n = 2**kn # ベクトルの長さ
    # B = 2**kB # ブロックの長さ

    # 初期サンプルをランダムに生成
    if seed != None:
        seed = int(seed)
        random.seed(seed)
    sample_number = numsamps * n
    sample_input_list = []

    for _ in range(sample_number):
        sample_input_list.append([Mod(random.randint(0, q), q) for _ in range(n)])

    # 各テストを順に実行し、かかった時間と結果を報告

    # Blind BKW with One Diff
    print("*************************************")
    print("Running Blind BKW with One Diff")

    blindfalse = blind_BKW(q, kn, B, sample_input_list, numsamps * n, alldiffs=False)
    timed_blindfalse = timeit("blindfalse.run()", number=1, repeat=1)
    print(timed_blindfalse)
    blindfalse.report()

  ############################################ 格納したいのだ ############################################
    import csv
    import datetime

    folder_path = '/Users/kenjiro/Documents/2_myj_lab/program/BKWonLWE/reduced_table/'
    exe_date = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

    csv_file_path = folder_path + 'reducedtable-' + exe_date + '.csv'

    # もっとうまく格納したい
    with open(csv_file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        for table in blindfalse.tables:
            for key, values in table.items():
                writer.writerow([key])
                for value in values:
                    writer.writerow(value)
                writer.writerow([])
        if show_final:
            blindfalse.show_final()
  ############################################ 格納したいのだ ############################################

    # Traditional BKW with One Diff
    print("*************************************")
    print("Running Traditional BKW with One Diff")
    tradfalse = trad_BKW(q, kn, B, sample_input_list, numsamps, alldiffs=False)
    timed_tradfalse = timeit("tradfalse.run()", number=1, repeat=1)
    print(timed_tradfalse)
    tradfalse.report()
    if show_final:
        tradfalse.show_final()

    if alldiffs:

        # Blind BKW with All Diffs
        print("*************************************")
        print("Running Blind BKW with All Diffs")
        blindtrue = blind_BKW(q, kn, B, sample_input_list, numsamps * n, alldiffs=True)
        timed_blindtrue = timeit("blindtrue.run()", number=1, repeat=1)
        print(timed_blindtrue)
        blindtrue.report()
        if show_final:
            blindtrue.show_final()

        # Traditional BKW with All Diffs
        print("*************************************")
        print("Running Traditional BKW with All Diff")
        tradtrue = trad_BKW(q, kn, B, sample_input_list, numsamps, alldiffs=True)
        timed_tradtrue = timeit("tradtrue.run()", number=1, repeat=1)
        print(timed_tradtrue)
        tradtrue.report()
        if show_final:
            tradtrue.show_final()


In [17]:
# Set up parameters
############### Experiment 1

# entries are mod q
q = 17

# dimension of vectors
kn = 4
n = 2^kn #length of vectors

# length of blocks
# kB = 2
# B = 2^kB # length of blocks
len_lastvec = 4
B = [4, 4, 2, 2, len_lastvec]

# number of samples
numsamps = 2000


# run experiment
run_experiment(q,kn,B,numsamps,alldiffs=False,seed=1,show_final=False)

*************************************
Running Blind BKW with One Diff
1 loop, best of 1: 825 ms per loop
Number of times a sample was passed to another table: 13528
Table 1 has 21538 entries.
Table 2 has 9295 entries.
Table 3 has 145 entries.
Table 4 has 145 entries.
Table 5 has 838 entries.
Total stored table rows (not counting final table): 31123
*************************************
Running Traditional BKW with One Diff
1 loop, best of 1: 949 ms per loop
Number of times a sample was passed to another table: 13845
Table 1 has 21441 entries.
Table 2 has 9319 entries.
Table 3 has 145 entries.
Table 4 has 144 entries.
Table 5 has 480 entries.
Total stored table rows (not counting final table): 31049


In [18]:
print(blindfalse.tables[5])

SortedDict({'[0, 0, 0, 2]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]], '[0, 0, 1, 7]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 7]], '[0, 0, 2, 7]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 7]], '[0, 0, 2, 9]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 9]], '[0, 0, 4, 6]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 6]], '[0, 0, 5, 6]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6]], '[0, 1, 0, 3]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3]], '[0, 1, 10, 10]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 10, 10]], '[0, 1, 12, 9]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 12, 9]], '[0, 1, 13, 14]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 13, 14]], '[0, 1, 13, 16]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 13, 16]], '[0, 1, 13, 7]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 13, 7]], '[0, 1, 14, 10]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 14, 10]], '[0, 1, 14, 14]': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 14, 14]], '[0, 1, 15, 